In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rioxarray
import rasterio
import matplotlib.pyplot as plt
import pyproj
import warnings
import os

from functools import partial
from shapely.ops import transform,cascaded_union
from keplergl import KeplerGl
from shapely.geometry import mapping,Point
from shapely.wkt import loads
from rasterio import plot


import sys
sys.path.insert(1, '../config/')
import kepler_configs

import pycrs


%matplotlib inline
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False
pd.options.display.max_columns = None

In [2]:
val_mapper = {
    0:'unknown',20:'shrubs',30:'herbacious_vegetation',40:'cultivated_cropland',50:'urban',60:'bare_sparse_vegetaion',70:'snow_ice',80:'water',
    90:'wetland_herbacious',100:'mosss',111:'forest_closed',112:'forest_closed',113:'forest_closed',114:'forest_closed',115:'forest_closed',
    116:'forest_closed',121:'forest_open', 122:'forest_open',123:'forest_open',124:'forest_open',125:'forest_open',126:'forest_open',200:'ocean'}



In [3]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]


In [4]:
def flip(x, y):
    """Flips the x and y coordinate values"""
    return y, x

In [5]:
tiff_path = '../data/raster/landcover/pak_landcover_100s.tif'

In [6]:
rds = rioxarray.open_rasterio(tiff_path, parse_coordinates=True)
district_shp_path  = '../data/vector/pak_administrative_shapefiles/District_Boundary.shp'
districts_gpd = gpd.read_file(district_shp_path)
# converting to EPSG:3857 as that is the projection for landcover raster
districts_gpd = districts_gpd.to_crs(3857)

In [7]:
poly = districts_gpd[districts_gpd.DISTRICT.isin(['JAMSHORO'])].geometry.values[0]
geom = mapping(loads(str(poly)))
dist_raster = rds.rio.clip([geom],'EPSG:3857',drop=True)
(unique, counts) = np.unique(dist_raster, return_counts=True)
dist_landcover = pd.DataFrame({'landcover':unique,'pixel_count':counts})
dist_landcover['landcover'] = dist_landcover.landcover.apply(lambda x: val_mapper[x])
def func_get_pct_of_known(df):
    df_known = df[~(df.landcover=='unknown')]
    total = df_known.pixel_count.sum()
    df_known['pct_of_known'] = df_known.pixel_count.apply(lambda x: (x/total)*100)
    return df_known
dist_landcover

,landcover,pixel_count
0,unknown,1288153
1,shrubs,71793
2,herbacious_vegetation,83157
3,cultivated_cropland,139788
4,urban,5068
5,bare_sparse_vegetaion,1068791
6,water,21470
7,wetland_herbacious,206
8,forest_closed,6
9,forest_open,2043


In [ ]:
districts = []
for district in districts_gpd.DISTRICT.unique():
    print(district)
    poly = districts_gpd[districts_gpd.DISTRICT.isin([district])].geometry.values[0]
    geom = mapping(loads(str(poly)))

    try:
        dist_raster = rds.rio.clip([geom],'EPSG:3857',drop=True)
        (unique, counts) = np.unique(dist_raster, return_counts=True)
        dist_landcover = pd.DataFrame({'landcover':unique,'pixel_count':counts})
        dist_landcover['landcover'] = dist_landcover.landcover.apply(lambda x: val_mapper[x])
        dist_landcover['district'] = district
        districts.append(dist_landcover)
    except:
        print('probably IOK')

BAGH
BHIMBER
HATTIAN BALA
HAVELI
KOTLI
MIRPUR
MUZAFFARABAD
NEELUM
POONCH
SUDHNOTI
AWARAN
BARKHAN
CHAGAI
DERA BUGTI
GWADAR
HARNAI
JAFFARABAD
JHAL MAGSI
KACHHI
KALAT
KECH
KHARAN
KHUZDAR
KILLA ABDULLAH
KILLA SAIFULLAH
KOHLU
LASBELA
LORALAI
MASTUNG
MUSA KHEL
NASIRABAD
NUSHKI
PANJGUR
PISHIN
QUETTA
SHEERANI
SIBI
WASHUK
ZHOB
ZIARAT
INDIAN OCCUPIED KASHMIR
BAJAUR AGENCY
FR BANNU
FR D.I.KHAN
FR KOHAT
FR LAKKI MARWAT
FR PESHAWAR
FR TANK
KURRAM AGENCY
MOHMAND AGENCY
N. WAZIRASTAN
ORAKZAI AGENCY
S. WAZIRASTAN
ISLAMABAD
ASTORE
DIAMIR
GHANCHE
probably IOK
GHIZER
GILGIT
HUNZA NAGAR
SKARDU
ABBOTTABAD
BANNU
BATAGRAM
BUNER
CHARSADDA
CHITRAL
D I KHAN
HANGU
HARIPUR
KARAK
KOHAT
UPPER KOHISTAN
LAKKI MARWAT
LOWER DIR
MALAKAND PROTECTED AREA
MANSEHRA
MARDAN
NOWSHERA
SHANGLA
SWABI
SWAT
TANK
TORDHER
UPPER DIR
ATTOCK
BAHAWALNAGAR
BAHAWALPUR
BHAKKAR
CHAKWAL
CHINIOT
D G KHAN
FAISALABAD
GUJRANWALA
GUJRAT
HAFIZABAD
JHANG
JHELUM
KASUR
KHUSHAB
LAHORE
LAYYAH
LODHRAN
MANDI BAHAUDDIN
MIANWALI
MULTAN
MUZAFARGARH
NANKANA S

In [1]:
districts_df = pd.concat(districts)

NameError: name 'pd' is not defined

In [69]:
districts_df.sort_values(by='pixel_count',ascending=False)

,landcover,pixel_count,district
0,unknown,1307986,NEELUM
0,unknown,428285,BHIMBER
0,unknown,424627,KOTLI
0,unknown,292851,MUZAFFARABAD
0,unknown,273996,POONCH
0,unknown,229484,MIRPUR
0,unknown,190900,BAGH
0,unknown,183402,HATTIAN BALA
0,unknown,145640,SUDHNOTI
0,unknown,113696,HAVELI


In [59]:
districts_df[districts_df.district=='KARACHI CENTRAL']

,landcover,pixel_count,district
0,unknown,5037,KARACHI CENTRAL
1,shrubs,13,KARACHI CENTRAL
2,herbacious_vegetation,21,KARACHI CENTRAL
3,cultivated_cropland,27,KARACHI CENTRAL
4,urban,7366,KARACHI CENTRAL
5,bare_sparse_vegetaion,68,KARACHI CENTRAL
6,forest_open,24,KARACHI CENTRAL


In [39]:
districts_df.to_csv('../data/summary-stats/landcover/landcover_pak.csv',index=False)